# DSCI 100 - Project Final Report
Date: June 23 2025

Author: Gwynnie Guo

## Introduction

### Background
This DSCI 100 project will investigate a question that arises from examining a real data science project conducted by a [research group in UBC](https://plai.cs.ubc.ca/) that has set up a Minecraft server to collect data about how players navigate the world.

### Question
This project will attempt to answer this broad question: **What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

In order to answer this broad question, I will begin by formulating a specific question from some of the variables in the dataset. My question will be: **Can player experience level, played hours, gender, and age predict whether or not the player will subscribe to a game-related newsletter in the dataset "players.csv", and how do these characteristics differ for players of differing experience levels?** 

### Data Description 
The dataset used for this project is the players.csv data, which is a list of all unique players, including data about each player. There are a total of 196 observations (players) and 7 variables. 

These are the variables in the dataset:

- `experience` - The experience level of each player. A factor data type going from Beginner, Amateur, Regular, Pro, Veteran.
- `subscribe` - Whether or not the player is subscribed to a game-related newsletter
- `hashedEmail` - The unique email/code for the player
- `played_hours`
- `name` - The name of the player
- `gender` - The gender of the player
- `Age` - The age of the player

# Read the dataset 
players <- read_csv("data/players.csv")
players